# Preamble

In this exercise we will look at how to run the GMPEs in OpenQuake's extensive GMPE library

In [ ]:
%matplotlib inline

# Import our usual Numpy and Matplotlib tools
import numpy as np
import matplotlib.pyplot as plt

## Which GMPEs are available to me?

If at any time you want to see which GMPEs are available on your version of OpenQuake you can just run this command.

There tell you the names you need to use when calling the GMPEs

In [ ]:
# Import the function to tell me which GMPEs are available
from openquake.hazardlib.gsim import get_available_gsims

# Print out the list
for gmpe in get_available_gsims():
    print(gmpe)

# <i>Everything in Context</i>

To make every GMPE run in the same way, OpenQuake needs you to organise your inputs into <b>source</b> parameters, <b>path (distance)</b> paramters and <b>site</b> parameters

<b>source</b> parameters refer to the properties of the rupture, e.g. the magnitude, the rake (or style-of faulting),
the top of rupture depth, hypocentral depth. These should always be scalar values (i.e. a single number)

<b>distances</b> refer to the source-to-site distance metrics (e.g. epicentral distsnce, hypocentral distance, Joyner-Boore distance etc. These must always be vector values (i.e. an array of numbers)

<b>site</b> refers to the site properties (e.g. $V_{S30}$, basin depth}, and should always be an array.

In [ ]:
from openquake.hazardlib.gsim.base import DistancesContext, SitesContext, RuptureContext

So, let's look at a simple example. We will take a simple GMPE (Bindi et al., 2014) and calculate the peak ground acceleration and 1.0 s spectral acceleration at a site 20 km Joyner-Boore distance from a $M_W$ 6.0 normal-faulting earthquakes, for a $V_{S30}$ of 500 m/s

In [ ]:
from openquake.hazardlib.gsim.bindi_2014 import BindiEtAl2014Rjb
from openquake.hazardlib.imt import PGA, SA
from openquake.hazardlib import const

#### Now set up the scenario

It is quite common to encounter a style-of-faulting term, e.g. normal, strike-slip, reverse.

OpenQuake requires that style of faulting is described by the rake angle of the fault, which follows this convention (Aki & Richards, 2002):

Normal = $-90^{\circ}$

Strike-slip = $0^{\circ}$ or $180^{\circ}$

Reverse = $90^{\circ}$

In [ ]:
rctx = RuptureContext()
rctx.mag = 6.0
rctx.rake = -90.0

dctx = DistancesContext()
dctx.rjb = np.array([20.0])

sctx = SitesContext()
sctx.vs30 = np.array([500.])

In [ ]:
gmpe = BindiEtAl2014Rjb()
imt = PGA()
median_pga, [stddev_pga] = gmpe.get_mean_and_stddevs(sctx, rctx, dctx, imt, [const.StdDev.TOTAL])
print("Median PGA = %.5f, Total Std. Dev = %.5f" % (np.exp(median_pga), stddev_pga))

In [ ]:
imt = SA(1.0)
median_sa1, [stddev_sa1] = gmpe.get_mean_and_stddevs(sctx, rctx, dctx, imt, [const.StdDev.TOTAL])
print("Median Sa(1.0) = %.5f, Total Std. Dev = %.5f" % (np.exp(median_sa1), stddev_sa1))

### How does the PGA attenuate with distance?

In [ ]:
dctx = DistancesContext()
dctx.rjb = np.arange(0., 201., 1.)

sctx = SitesContext()
sctx.vs30 = 500. * np.ones_like(dctx.rjb) 

In [ ]:
imt = PGA()
median_pga, [stddev_pga] = gmpe.get_mean_and_stddevs(sctx, rctx, dctx, imt, [const.StdDev.TOTAL])
plt.figure()
plt.plot(dctx.rjb, np.exp(median_pga), "k-", lw=2)
plt.plot(dctx.rjb, np.exp(median_pga + stddev_pga), "r--")
plt.plot(dctx.rjb, np.exp(median_pga - stddev_pga), "r--")
plt.grid()

### How does the GMPE vary with Spectral Period (for 1st scenario)?

In [ ]:
dctx = DistancesContext()
dctx.rjb = np.array([20.0])

sctx = SitesContext()
sctx.vs30 = np.array([500.])

periods = np.logspace(np.log10(0.05), np.log10(3.0), 100)


In [ ]:
median_pga = np.empty(len(periods))
stddev_pga = np.empty(len(periods))

# Loop over each period
for i, period in enumerate(periods):
    median, [stddev] = gmpe.get_mean_and_stddevs(sctx, rctx, dctx, SA(period), [const.StdDev.TOTAL])
    median_pga[i] = median[0]
    stddev_pga[i] = stddev[0]

# Plot the results
plt.figure()
plt.semilogx(periods, np.exp(median_pga), "k-", lw=2)
plt.semilogx(periods, np.exp(median_pga + stddev_pga), "r--")
plt.semilogx(periods, np.exp(median_pga - stddev_pga), "r--")
plt.grid(True)
plt.xlim(0.05, 3.0)

### Now it's your turn ...

Can you do the following?

1. Show how the PGA changes with magnitude for the same site (20 km from the earthquake source with Vs30 500 m/s)

2. Show how the PGA changes with Vs30 for the Mw 6.0 earthquake at a site 20 km from the normal faulting source

# What's the damage?

You are building a new building at the site that is 20 km from the normal fault that can produce a Mw 6.5 earthquake, with the Vs30 = 300 m/s. The engineer tells you that the building can withstand 0.15 g without sustaining any damage, but at 0.5 g it will collapse!

If the earthquake were to occur, what would be the probability that the building:
1. sustains no damage
2. collapses

In [ ]:
# Import a Scipy tool for the normal distribution
from scipy.stats import norm

In [ ]:
rctx = RuptureContext()
rctx.mag = 6.5
rctx.rake = -90.0

dctx = DistancesContext()
dctx.rjb = np.array([20.0])

sctx = SitesContext()
sctx.vs30 = np.array([300.])

In [ ]:
imt = PGA()
gmpe = BindiEtAl2014Rjb()
median_pga, [stddev] = gmpe.get_mean_and_stddevs(sctx, rctx, dctx, imt, [const.StdDev.TOTAL])
print("Median PGA = %.5f and Std. Deviation = %.4f" % (np.exp(median_pga), stddev))

Consider the "no damage" case - how many standard deviations above or below the median is this 0.15 g threshold?

In [ ]:
accel = np.logspace(-3., 1., 1000.)
cdf = norm.cdf(np.log(accel), loc=median_pga[0], scale=stddev[0])
plt.figure(figsize=(8,8))
plt.semilogx(accel, cdf, "r-")
plt.grid()
plt.xlabel("g", fontsize=16)
plt.ylabel("Probability", fontsize=16)

In [ ]:
# Number of standard deviations above/below mean
num_stddevs = (np.log(0.15) - median_pga) / stddev
print("0.15 g is %.3f standard deviations above the median" % num_stddevs)
prob_no_damage = norm.cdf(num_stddevs, loc=0., scale=1.)
print("The building has a %.4f probability of observing no damage" % prob_no_damage)

### What is the probability of collapse?

In [ ]:
num_stddevs = (np.log(0.5) - median_pga) / stddev
print("0.5 g is %.3f standard deviations above the median" % num_stddevs)
# Note the use of the "survival function [sf]": 1.0 - cdf
prob_collapse = norm.sf(num_stddevs, loc=0., scale=1.)
print("The building has a %.4f probability of collapsing" % prob_collapse)